In [2]:
#Step 1: recieve an observation
import gym


env = gym.make('CartPole-v1')

PARAM_episode_max_length = 100

observation = env.reset()

# for t in range(PARAM_episode_max_length):

observation

(array([ 0.03998628, -0.04274393, -0.00119471,  0.00502334], dtype=float32),
 {})

In [2]:
#Step 2: run an episode of environment
import gym


env = gym.make('CartPole-v1', render_mode='human')

PARAM_episode_max_length = 100

observation = env.reset()

for t in range(PARAM_episode_max_length):
    random_action = env.action_space.sample()
    observation, reward, is_terminated, is_truncated, step_info = env.step(random_action)

env.close()

In [3]:
env.observation_space.shape

(4,)

In [1]:
#Step 3: Define, Implement the state space of the Q Algorithm

#The state space is a superset of the observation spae

#observations consists of 4 continuous components

#There is no way I can think of to implement a continuous state space
# without discretizing it, and thus states within the state space 
# of the  Q-learning algorithm will be represented by ranges.
import gym, math


env = gym.make('CartPole-v1')

# Run 1000 episodes to determine observed boundary values for each component 
PARAM_number_of_episodes = 1000
PARAM_episode_max_length = 1000

state_space_num_components = env.observation_space.shape[0]

class state_space_representation:
    def __init__(self, num_components):
        self.num_components = num_components
        self.min_max_bounds = [[float('inf'), float('-inf')] for i in range(self.num_components)]
        self.repr_created = False

    def update(self, observation):
        #dont update if repr_created
        assert not (self.repr_created == True)

        for i in range(self.num_components):
            if observation[i] < self.min_max_bounds[i][0]:
                self.min_max_bounds[i][0] = observation[i]
            if observation[i] > self.min_max_bounds[i][1]:
                self.min_max_bounds[i][1] = observation[i]
        return

    def create_state_space_repr(self):
        #The state space is represented as a partition
        # of interval (-inf, inf) into a number of 
        # subintervals.
        # The number of intervals, and the size of 
        # each interval is determined by the length of the
        # difference of the most extreme observed values of
        # each component of the observation space.
        self.state_space_component_partitions  = []

        #The larger this number, the more elements will exist
        # within the partition of each state space component.
        self.state_space_component_complexitity_multiplier = 4
        self.ssccm = self.state_space_component_complexitity_multiplier

        for i in range(self.num_components):
            #Given that this code was written for the cartpole environment,
            # the intervals of each component of the cartpole observation space 
            # are all larger than 1.
            # Thus the number of elements in each partition will always al least
            # at least the ssccm. 
            number_of_elements_in_partition = math.ceil(self.interval_lengths[i] * self.ssccm)
            low, high = self.min_max_bounds[i]

            partition = [(-1 * float('inf'), low) ]
            
            interval_step_size = self.interval_lengths[i] / number_of_elements_in_partition 

            for j in range(number_of_elements_in_partition):
                interval_low = low + (j * interval_step_size)
                interval_high = low + ((j + 1) * interval_step_size)
                interval = (interval_low, interval_high)
                partition.append(interval)

            #Should be true, but a guarantee was not proven  
            assert (len(partition) > 0)
            
            print((partition[-1][1], high, partition[-1][1] - high))

            partition.append((partition[-1][1], float('inf')))
            self.state_space_component_partitions.append(partition)
        return

    def finalize(self):
        self.repr_created = True
        self.interval_lengths = [self.min_max_bounds[i][1] - self.min_max_bounds[i][0] for i in range(self.num_components)]
        self.create_state_space_repr()
        return 
    
    


In [2]:
import gym, math


env = gym.make('CartPole-v1')

# Run 1000 episodes to determine observed boundary values for each component 
PARAM_number_of_episodes = 1000
PARAM_episode_max_length = 1000

state_space_num_components = env.observation_space.shape[0]

state_space_repr = state_space_representation(state_space_num_components)

for episode in range(PARAM_number_of_episodes):
    observation, _ = env.reset()
    state_space_repr.update(observation)

    for t in range(PARAM_episode_max_length):
        random_action = env.action_space.sample()
        observation, reward, terminate_episode_signal, _, step_info = env.step(random_action)
        state_space_repr.update(observation)
        
        if terminate_episode_signal:
            break


state_space_repr.finalize()

c:\Users\tareg\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\classic_control\cartpole.py:187: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


(2.490264892578125, 2.4902651, -2.384185791015625e-07)
(5.126671314239502, 5.1266713, 0.0)
(127.28125, 127.281265, -1.52587890625e-05)
(19.483659744262695, 19.48366, 0.0)


In [3]:
#Step 3: Define, Implement the state space of the Q Algorithm

#The state space is a superset of the observation spae

#observations consists of 4 continuous components

#There is no way I can think of to implement a continuous state space
# without discretizing it, and thus states within the state space 
# of the  Q-learning algorithm will be represented by ranges.
import gym


env = gym.make('CartPole-v1')

# Run 1000 episodes to determine observed boundary values for each component 
PARAM_number_of_episodes = 1000
PARAM_episode_max_length = 1000

state_space_num_components = env.observation_space.shape[0]

#(min, max) observed for each component 
state_space_repr_obj = [[float('inf'), float('-inf')] for i in range(state_space_num_components)]



episode_length_history = []

for episode in range(PARAM_number_of_episodes):
    observation, _ = env.reset()
    
    for i in range(state_space_num_components):
        if observation[i] < state_space_repr_obj[i][0]:
            state_space_repr_obj[i][0] = observation[i]
        if observation[i] > state_space_repr_obj[i][1]:
            state_space_repr_obj[i][1] = observation[i]


    terminated_at_counter = 0 

    for t in range(PARAM_episode_max_length):
        random_action = env.action_space.sample()
        observation, reward, terminate_episode_signal, _, step_info = env.step(random_action)

        for i in range(state_space_num_components):
            if observation[i] < state_space_repr_obj[i][0]:
                state_space_repr_obj[i][0] = observation[i]
            if observation[i] > state_space_repr_obj[i][1]:
                state_space_repr_obj[i][1] = observation[i]

        if not (terminate_episode_signal):
            terminated_at_counter += 1
        else:
            break

    episode_length_history.append(terminated_at_counter)

# len(observations)

# observed_bounds = []
state_space_repr_obj
# for i in range(state_space_num_components):
#     observed_min = min(observations, key=lambda x: x[i])[i]
#     observed_max = max(observations, key=lambda x: x[i])[i]
#     observed_bounds.append((observed_min, observed_max))

# observed_bounds, sum(episode_length_history)/len(episode_length_history)

[[-2.4893837, 2.4896784],
 [-5.4872537, 5.069546],
 [-100.3937, 116.41859],
 [-17.804838, 19.29232]]

In [4]:
interval_lengths = [(high - low) for low, high in observed_bounds]
interval_lengths

[4.96159, 9.648396, 252.36862, 38.21286]

In [24]:
#Divide each component into n times the interval length
import math

state_space_component_partitions  = []

#The larger this number, the more elements will exist
# within the partition of each state space component.
state_space_component_complexitity_multiplier = 4
ssccm = state_space_component_complexitity_multiplier

for i in range(len(interval_lengths)):
    #Given that this code was written for the cartpole environment,
    #the intervals of each component of the cartpole obs. space are all larger
    # than 1.
    # Thus the number of elements in each partition will always al least
    # at least the ssccm. 
    number_of_elements_in_partition = math.ceil(interval_lengths[i] * ssccm)
    low, high = observed_bounds[i]

    partition = [(-1 * float('inf'), low) ]
    
    interval_step_size = interval_lengths[i] / number_of_elements_in_partition 

    for j in range(number_of_elements_in_partition):
        interval_low = low + (j * interval_step_size)
        interval_high = low + ((j + 1) * interval_step_size)
        interval = (interval_low, interval_high)
        partition.append(interval)

    #Should be true, but a guarantee was not proven  
    assert (len(partition) > 0)
    
    print((partition[-1][1], high, partition[-1][1] - high))

    partition.append((partition[-1][1], float('inf')))
    state_space_component_partitions.append(partition)

state_space_component_partitions[0]


(2.481074810028076, 2.4810748, 0.0)
(4.868475914001465, 4.8684754, 4.76837158203125e-07)
(112.92225646972656, 112.92225, 7.62939453125e-06)
(18.531221389770508, 18.531221, 0.0)


[(-inf, -2.480515),
 (-2.4805150032043457, -2.2324355125427244),
 (-2.2324355125427244, -1.9843560218811036),
 (-1.9843560218811036, -1.7362765312194823),
 (-1.7362765312194823, -1.4881970405578613),
 (-1.4881970405578613, -1.2401175498962402),
 (-1.2401175498962402, -0.992038059234619),
 (-0.992038059234619, -0.7439585685729979),
 (-0.7439585685729979, -0.49587907791137686),
 (-0.49587907791137686, -0.2477995872497556),
 (-0.2477995872497556, 0.0002799034118652344),
 (0.0002799034118652344, 0.2483593940734865),
 (0.2483593940734865, 0.4964388847351078),
 (0.4964388847351078, 0.7445183753967286),
 (0.7445183753967286, 0.9925978660583499),
 (0.9925978660583499, 1.2406773567199707),
 (1.2406773567199707, 1.488756847381592),
 (1.488756847381592, 1.7368363380432132),
 (1.7368363380432132, 1.9849158287048345),
 (1.9849158287048345, 2.232995319366455),
 (2.232995319366455, 2.481074810028076),
 (2.481074810028076, inf)]

In [25]:
[len(partition) for partition in state_space_component_partitions]

[22, 41, 1012, 155]

In [3]:
[len(partition) for partition in state_space_repr.state_space_component_partitions]

[22, 44, 1255, 158]

In [26]:
g = 0
for i in range(144400000):
    g += 1

In [52]:
import random

class Q:
    def __init__(self, state_space_repr, action_space_repr, learning_rate, discount_factor, epsilon, q_init_value):
        self.state_space_repr = state_space_repr
        self.action_space_repr = action_space_repr
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.q_init_value = q_init_value
        self.q_table = {}
        self.state_lookup_misses = 0
        # self.delta_history = []
        return

    def map_observation_to_repr(self, observation):
        #Given an observation, return the state space representation
        # that it belongs to.
        assert (len(observation) == self.state_space_repr.num_components)
        repr = []

        for i in range(self.state_space_repr.num_components):
            for j in range(len(self.state_space_repr.state_space_component_partitions[i])):
                low, high = self.state_space_repr.state_space_component_partitions[i][j]
                if observation[i] >= low and observation[i] < high:
                    repr.append(j)
                    break
        repr = tuple(repr)


        return repr

    def best_action(self, observation):
        repr = self.map_observation_to_repr(observation)
        if repr not in self.q_table:
            self.state_lookup_misses += 1
            #place current state in table
            self.q_table[repr] = {i:self.q_init_value for i in range(self.action_space_repr.n)}
        
        if random.random() <= self.epsilon:
            return self.action_space_repr.sample()

        all_action_values_equal = True
        best_action = 0
        best_value = self.q_init_value
        
        for action in self.q_table[repr]:
            value = self.q_table[repr][action] 
            if (value > best_value):
                all_action_values_equal = False
                best_action = action
                best_value = value

        if all_action_values_equal:
            return self.action_space_repr.sample()
        else:
            return best_action
    
    def update(self, observation, action, reward, next_observation):
        repr = self.map_observation_to_repr(observation)
        next_repr = self.map_observation_to_repr(next_observation)

        if repr not in self.q_table:
            #place current state in table
            self.state_lookup_misses += 1
            self.q_table[repr] = {i:self.q_init_value for i in range(self.action_space_repr.n)}

        if next_repr not in self.q_table:
            #place next state in table
            self.state_lookup_misses += 1
            self.q_table[next_repr] = {i:self.q_init_value for i in range(self.action_space_repr.n)}

        
        #update q value
        old_value = self.q_table[repr][action]
        current_value_information = (1 - self.learning_rate) * self.q_table[repr][action]
        temporal_diff_target = self.learning_rate * (reward + self.discount_factor * max(self.q_table[next_repr].values()) )
        self.q_table[repr][action] = current_value_information + temporal_diff_target  

        # delta = self.q_table[repr][action] - old_value
        # print( (old_value, self.q_table[repr][action], delta, temporal_diff_target), sep="", end=" " )
        # self.delta_history.append(delta)
        return

In [5]:
mean_reward_per_session = []

In [56]:
# Step 5: Run Algorithm, Achieve Convergence
# Code should take in an (observation, action), determine the 
# bounding interval for the observation.
# If a state has not been accessed before, it's 
# pre-updated Q value is 0.
env = gym.make('CartPole-v1', render_mode='human')

# Run 1000 episodes to determine observed boundary values for each component 
PARAM_number_of_episodes = 30
PARAM_episode_max_length = 1000

PARAM_learning_rate = 0.8
PARAM_discount_factor = 0.8

table = q.q_table

q = Q(
    state_space_repr=state_space_repr, 
    action_space_repr=env.action_space, 
    learning_rate=PARAM_learning_rate, 
    discount_factor=PARAM_discount_factor,
    epsilon=0.2,
    q_init_value=0
)

q.q_table = table

import pickle 

with open('q-table.pkl', 'wb') as f:
    pickle.dump(q.q_table, f)


In [53]:
# Step 5: Run Algorithm, Achieve Convergence
# Code should take in an (observation, action), determine the 
# bounding interval for the observation.
# If a state has not been accessed before, it's 
# pre-updated Q value is 0.
import gym

env = gym.make('CartPole-v1')

PARAM_number_of_episodes = 100
PARAM_episode_max_length = 1000

PARAM_learning_rate = 0.8
PARAM_discount_factor = 0.8

table = q.q_table

q = Q(
    state_space_repr=state_space_repr, 
    action_space_repr=env.action_space, 
    learning_rate=PARAM_learning_rate, 
    discount_factor=PARAM_discount_factor,
    epsilon=0.2,
    q_init_value=0
)

q.q_table = table

import pickle 

with open('q-table.pkl', 'wb') as f:
    pickle.dump(q.q_table, f)

In [57]:
reward_history = []

for episode in range(PARAM_number_of_episodes):
    observation, _ = env.reset()

    episode_reward = 0
    misses_at_start = q.state_lookup_misses
    for t in range(PARAM_episode_max_length):
        env.render()
        action = q.best_action(observation)
        next_observation, reward, terminate_episode_signal, _, step_info = env.step(action)
        q.update(observation, action, reward, next_observation)
        
        episode_reward += reward
        observation = next_observation

        if terminate_episode_signal:
            break
    
    reward_history.append(episode_reward)

    misses_during_episode = q.state_lookup_misses - misses_at_start
    episodes_left = PARAM_number_of_episodes - episode - 1
    exp_info = (episodes_left, len(q.q_table), misses_during_episode, episode_reward) 
    print(exp_info)

env.close()

import numpy as np
import win32api

session_reward = np.mean(reward_history)
mean_reward_per_session.append(session_reward)


win32api.MessageBox(0, 'Done', 'DoneMessage', 0x00001000) 

mean_reward_per_session

(29, 240614, 19, 54)
(28, 240677, 63, 22)
(27, 240726, 49, 9)
(26, 240864, 138, 42)
(25, 240953, 89, 37)
(24, 241380, 427, 44)
(23, 241559, 179, 37)
(22, 241784, 225, 12)
(21, 241806, 22, 35)
(20, 241865, 59, 31)
(19, 241922, 57, 32)
(18, 242088, 166, 24)
(17, 242122, 34, 29)
(16, 242286, 164, 54)
(15, 242338, 52, 46)
(14, 242437, 99, 20)
(13, 242542, 105, 24)
(12, 242607, 65, 26)
(11, 242658, 51, 7)
(10, 242737, 79, 19)
(9, 242774, 37, 13)
(8, 242822, 48, 27)
(7, 242861, 39, 33)
(6, 242893, 32, 27)
(5, 242968, 75, 15)
(4, 243002, 34, 71)


: 

: 

In [55]:
mean_reward_per_session

[9.885714285714286,
 23.9,
 20.3,
 12.1,
 9.9,
 17.3,
 12.2,
 13.86,
 18.6,
 13.066666666666666,
 11.99,
 18.67,
 35.5,
 28.82,
 30.29]

In [6]:
import pickle 
with open('q-table.pkl', 'rb') as f:
    table = pickle.load(f)